In [35]:
import urllib
import pandas as pd 
from sunpy.time import TimeRange
from flarelist_utils import read_swpc_reports, read_ngdc_goes_reports
from dateutil.relativedelta import relativedelta
import pandas as pd 
import numpy as np
import datetime
import glob
import warnings
warnings.filterwarnings("ignore")

In [36]:
def print_flares(df):
    x = np.sum(df["goes_class_ind"].isin(["X"]))
    m = np.sum(df["goes_class_ind"].isin(["M"]))
    c = np.sum(df["goes_class_ind"].isin(["C"]))
    print("X: {:d}, M: {:d}, C: {:d}".format(x, m, c))

In [41]:
def get_yearly_swpc(year):
    year = datetime.datetime(year, 1, 1)
    filedir = "./goes_files/%Y_events/%Y*events*"
    all_files = []
    all_files += glob.glob(year.strftime(filedir))
    all_files.sort()
    df_flares = read_swpc_reports(all_files[0])
    for f in all_files[1:]:
        df = read_swpc_reports(f)
        df_flares = df_flares.append(df)
    df_flares.reset_index(inplace=True, drop=True)
    df_flares["ts"] = df_flares.date + df_flares.start_time
    df_flares.drop_duplicates(subset="ts")
    return df_flares[df_flares["goes_class_ind"].isin(["C", "X", "M"])]


def get_yearly_ngdc(year):
    year = datetime.datetime(year, 1, 1)
    file = "goes-xrs-report_%Y.txt"
    df = read_ngdc_goes_reports(year.strftime(file))
    df["ts"] = df.date + df.start_time
    df.drop_duplicates(subset="ts")
    return df[df["goes_class_ind"].isin(["X", "M", "C"])]

def get_yearly_ssw(year):
    tstart = datetime.datetime(year, 1, 1)
    tend = datetime.datetime(year, 12, 31)
    return ssw[(ssw["datetime"]>=tstart) & (ssw["datetime"]<=tend)]
    

In [42]:
ssw_flarelist = pd.read_csv("full_sswlatest.csv")
ssw = ssw_flarelist[ssw_flarelist["goes_class_ind"].isin(["X", "M", "C"])]
ssw["datetime"] = pd.to_datetime(ssw["time_start"])

In [43]:
print("SSW from helio:"); print_flares(ssw)

SSW from helio:
X: 49, M: 735, C: 7989


In [47]:
def check_for_years(year):
    print(year)
    print_flares(get_yearly_ssw(year))
    print_flares(get_yearly_ngdc(year))
    print_flares(get_yearly_swpc(year))

In [48]:
for i in [2010, 2011, 2012, 2013, 2014, 2015, 2016]:
    check_for_years(i)

2010
X: 0, M: 19, C: 161
X: 0, M: 23, C: 170
X: 0, M: 23, C: 170
2011
X: 8, M: 105, C: 1254
X: 8, M: 111, C: 1200
X: 8, M: 111, C: 1200
2012
X: 7, M: 112, C: 1216
X: 7, M: 123, C: 1259
X: 7, M: 129, C: 1337
2013
X: 12, M: 96, C: 1417
X: 12, M: 99, C: 1356
X: 12, M: 98, C: 1349
2014
X: 16, M: 210, C: 1906
X: 16, M: 205, C: 1797
X: 16, M: 207, C: 1793
2015
X: 2, M: 132, C: 1411
X: 2, M: 104, C: 1301
X: 2, M: 125, C: 1375
2016
X: 0, M: 17, C: 343
X: 0, M: 16, C: 302
X: 0, M: 16, C: 321


In [49]:
year = 2016
ssw_test = get_yearly_ssw(year)
ngdc_test = get_yearly_ngdc(year)
swpc_test = get_yearly_swpc(year)

In [59]:
ssw_test_m = ssw_test[ssw_test["goes_class_ind"].isin(["M"])]; ssw_test_m.reset_index(inplace=True, drop=True)
ngdc_test_m = ngdc_test[ngdc_test["goes_class_ind"].isin(["M"])]
swpc_test_m = swpc_test[swpc_test["goes_class_ind"].isin(["M"])]

In [72]:
ssw_test_m.iloc[8:12]

,time_start,time_end,time_peak,xray_class,lat_hg,long_hg,long_carr,nar,ename,url_nar,url_flare,goes_class_ind,goes_class_val,datetime
8,2016-07-23 01:40:00.0,2016-07-23 05:45:00.0,2016-07-23 05:16:00.0,M7.6,5,73,173.32,12565.0,gev_20160723_0140,http://www.solarmonitor.org/region.php?date=20...,http://www.lmsal.com/solarsoft/latest_events_a...,M,0.000076,2016-07-23 01:40:00
9,2016-07-23 01:46:00.0,2016-07-23 02:23:00.0,2016-07-23 02:11:00.0,M5.0,5,73,173.27,12567.0,gev_20160723_0146,http://www.solarmonitor.org/region.php?date=20...,http://www.lmsal.com/solarsoft/latest_events_a...,M,0.000050,2016-07-23 01:46:00
10,2016-07-23 05:00:00.0,2016-07-23 05:24:00.0,2016-07-23 05:16:00.0,M7.6,5,73,171.49,12567.0,gev_20160723_0500,http://www.solarmonitor.org/region.php?date=20...,http://www.lmsal.com/solarsoft/latest_events_a...,M,0.000076,2016-07-23 05:00:00
11,2016-07-23 05:27:00.0,2016-07-23 05:33:00.0,2016-07-23 05:31:00.0,M5.5,2,75,173.24,12565.0,gev_20160723_0527,http://www.solarmonitor.org/region.php?date=20...,http://www.lmsal.com/solarsoft/latest_events_a...,M,0.000055,2016-07-23 05:27:00


In [75]:
swpc_test_m.iloc[7:11]

,date,event_no,start_time,max_time,end_time,goes_sat,goes_channel,goes_class_ind,goes_class,integrated_flux,noaa_ar,ts
816,20160721,4680,0134,0149,0204,G15,1-8A,M,M1.0,1.2E-02,12567,201607210134
842,20160723,5430,0146,0211,0223,G15,1-8A,M,M5.0,5.4E-02,12567,201607230146
844,20160723,5450,0500,0516,0524,G15,1-8A,M,M7.6,4.6E-02,12567,201607230500
845,20160723,5470,0527,0531,0533,G15,1-8A,M,M5.5,1.1E-02,12567,201607230527
